In [1]:
import numpy as np

from langevian_protein import run
from langevian_protein import calculate_configuration_force

In [3]:
protein_chain = ['H', 'P', 'P', 'H', 'H']
coordinates = np.array([
    [0,0,1],
    [0,0,2],
    [0,0,3],
    [0,0,4],
    [0,0,5]
])
friction = 0.05
m = 1
kT = 2
dt = 0.003
k = 20
l = 1
iterations = 10

In [4]:
calculate_configuration_force(coordinates, protein_chain, k, l)

(-0.13315560198848553,
 array([[ 0.        ,  0.        ,  0.13350171],
        [ 0.        ,  0.        ,  0.12838964],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        , -0.13203758],
        [ 0.        ,  0.        , -0.12985377]]))

In [ ]:
# TODO: Go step wise from here and figure out the wrong step - we have the data for one correct iteration.